# ANNDL - Challenge 1

### Importation

In [ ]:
import os
import random
import matplotlib.pyplot as plt
import PIL
import PIL.Image

#updated tensorflow's version to use the ConvNeXt models
!pip install --upgrade tensorflow    

import tensorflow as tf
import cython as ct
import numpy as np
import matplotlib as mpl
import seaborn as sns
import scipy as sp
import sklearn as sk
import pandas as pd

tfk = tf.keras
tfkl = tf.keras.layers

print('Tensorflow: ', tf.__version__)
print('Cython: ', ct.__version__)
print('Numpy: ', np.__version__)
print('Matploit: ', mpl.__version__)
print('Seaborn: ', sns.__version__)
print('Scipy: ', sp.__version__)
print('Scikit-learn: ', sk.__version__)
print('Panda: ', pd.__version__)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 588.3 MB 6.7 kB/s 
     |████████████████████████████████| 6.0 MB 48.7 MB/s 
     |████████████████████████████████| 1.7 MB 56.3 MB/s 
     |████████████████████████████████| 439 kB 69.7 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully 

### Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/MyDrive/'Submission'

Mounted at /gdrive
/gdrive/.shortcut-targets-by-id/1wYXyoEwRqU1umxccIJJH4SamZKzkPV7x/ANNDL/Challenge1


### Turn off Tensorflow warnings

In [ ]:
import warnings
import logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)
tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.get_logger().setLevel('ERROR')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

### Set reproducibility seed

In [ ]:
seed = 42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

### Set parameters

In [ ]:
image_size = 224 #Resizing image  
input_shape = (image_size, image_size, 3)
epochs = 200
neural_network_name = 'convnext_1'
batch_size = 32

### Setting up image generator
The image generator is useful for creating a set of modifications for the images. However data it'not loaded yet.
Images are divided into 8 folders, one for each class, so we can exploit the ImageDataGenerator to read them from disk.

In [ ]:
from keras.applications.convnext import preprocess_input

train_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    # apply the net preprocessing to our data
    preprocessing_function=preprocess_input,
    validation_split = 0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='reflect',
    brightness_range = [0.7, 1.5],
    rotation_range = 30,
    shear_range = 0.3
)

### Data loading with ImageGenerator

In [ ]:
# Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method
#for train and validation sets

train_set = train_gen.flow_from_directory(
    directory=('Images/Dataset'),
    target_size=(image_size, image_size),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=seed,
    subset = 'training'
)

val_set = train_gen.flow_from_directory(
    directory=('Images/Dataset'),
    target_size=(image_size,image_size),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size,
    classes=None,
    shuffle=True,
    subset = 'validation',
    seed=seed
)


Found 2836 images belonging to 8 classes.
Found 706 images belonging to 8 classes.


### Callbacks

In [ ]:
# Utility function to create folders and callbacks for training
from datetime import datetime

def create_folders_and_callbacks(model_name):

  exps_dir = os.path.join('Federica/models_TL_pat')
  if not os.path.exists(exps_dir):
      os.makedirs(exps_dir)

  now = datetime.now().strftime('%b%d_%H-%M-%S')

  exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
  if not os.path.exists(exp_dir):
      os.makedirs(exp_dir)
      
  callbacks = []

  # Model checkpoint
  # ----------------
  ckpt_dir = os.path.join(exp_dir, 'ckpts')
  if not os.path.exists(ckpt_dir):
      os.makedirs(ckpt_dir)

  ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp.ckpt'), 
                                                     save_weights_only=False, # True to save only weights
                                                     save_best_only=True) # True to save only the best epoch 
  callbacks.append(ckpt_callback)

  # Visualize Learning on Tensorboard
  # ---------------------------------
  tb_dir = os.path.join(exp_dir, 'tb_logs')
  if not os.path.exists(tb_dir):
      os.makedirs(tb_dir)
      
  # By default shows losses and metrics for both training and validation
  tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir, 
                                               profile_batch=0,
                                               histogram_freq=1)  # if > 0 (epochs) shows weights histograms
  callbacks.append(tb_callback)

  # Early Stopping
  # --------------
  es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True)
  callbacks.append(es_callback)

  return callbacks

### Transfer Learning

In [ ]:
#ConvNeXtLarge model 
supernet = tfk.applications.ConvNeXtXLarge(
    include_top=False,
    weights="imagenet",
    input_shape=input_shape   
)
supernet.summary()
#tfk.utils.plot_model(supernet)

In [ ]:
# Use the supernet as feature extractor
supernet.trainable = False

tl_model = tfk.models.Sequential()
tl_model.add(tfkl.Resizing(image_size, image_size, interpolation="bicubic"))
tl_model.add(supernet)
tl_model.add(tfkl.GlobalAveragePooling2D())
tl_model.add(tfkl.Dropout(0.3))
tl_model.add(tfkl.Dense(3072,activation='relu',kernel_initializer='he_normal'))
tl_model.add(tfkl.Dense(8,activation='softmax',kernel_initializer='glorot_normal'))

# Compile the model
tl_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-5), metrics='accuracy')
tl_model.build([None, image_size,image_size,3])
tl_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing (Resizing)         (None, 224, 224, 3)       0         
                                                                 
 convnext_xlarge (Functional  (None, 7, 7, 2048)       348147968 
 )                                                               
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 3072)              6294528   
                                                                 
 dense_1 (Dense)             (None, 8)                 2

In [ ]:
# Train the model
tl_history = tl_model.fit( 
    x = train_set,
    batch_size = batch_size,
    epochs = 100,
    validation_data = val_set,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True)]
).history  

In [ ]:
#evaluate the model using the val_set
model_metrics = tl_model.evaluate(val_set, return_dict=True)  

### Fine tuning

In [ ]:
# Set all layers to True
tl_model.get_layer('convnext_xlarge').trainable = True

#The only built-in layer that has non-trainable weights is the BatchNormalization layer
for i, layer in enumerate(tl_model.get_layer('convnext_xlarge').layers):
  if(layer.__class__.__name__ == 'LayerNormalization'):
    layer.trainable = False

for i, layer in enumerate(tl_model.get_layer('convnext_xlarge').layers):
   print(i, layer.name, layer.__class__.__name__, layer.trainable)
tl_model.summary()

In [ ]:
tl_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-5), metrics='accuracy')
# Fine-tune the model
ft_history = tl_model.fit(
    x = train_set,
    batch_size = batch_size,
    epochs = 100,
    validation_data = val_set,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True)],
).history

In [ ]:
# Plot the training
plt.figure(figsize=(15,5))
plt.plot(ft_history['loss'], alpha=.3, label='Training', color='#ff7f0e', linestyle='--')
plt.plot(ft_history['val_loss'], label='Validation Fine Tuning', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(ft_history['accuracy'], label='Training', alpha=.8, color='#ff7f0e', linestyle='--')
plt.plot(ft_history['val_accuracy'], label='Validation Fine Tuning', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

In [ ]:
# Save best epoch model
tl_model.save('Submission/FT_convnextx_large',include_optimizer=False)
final_model = tfk.models.load_model('Submission/FT_convnextx_large')

### Results

In [ ]:
model1_metrics = tl_model.evaluate(val_set, return_dict=True)